In [2]:
!pip install snowflake['ml']

     |████████████████████████████████| 450kB 1.3MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 35.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 48.0MB/s eta 0:00:01
     |████████████████████████████████| 122kB 56.3MB/s eta 0:00:01
     |████████████████████████████████| 409kB 37.4MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 32.6MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 45.9MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 280kB/s  eta 0:00:01
     |████████████████████████████████| 12.2MB 46.5MB/s eta 0:00:01
     |████████████████████████████████| 747kB 43.3MB/s eta 0:00:01
     |████████████████████████████████| 41.0MB 273kB/s  eta 0:00:01
     |████████████████████████████████| 2.5MB 30.6MB/s eta 0:00:01
     |████████████████████████████████| 51kB 8.9MB/s  eta 0:00:01
     |████████████████████████████████| 81kB 12.3MB/s eta 0:00:01
     |████████████████████████████████| 133kB 41.3MB/s eta 0:

     |████████████████████████████████| 2.1MB 31.0MB/s eta 0:00:01     |████████████████████████████████| 2.1MB 31.0MB/s 
     |████████████████████████████████| 71kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 870kB 24.2MB/s eta 0:00:01
     |████████████████████████████████| 307kB 52.9MB/s eta 0:00:01
     |████████████████████████████████| 512kB 33.8MB/s eta 0:00:01
     |████████████████████████████████| 81kB 12.5MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 24.8MB/s eta 0:00:01
     |████████████████████████████████| 450kB 41.2MB/s eta 0:00:01
     |████████████████████████████████| 61kB 10.0MB/s eta 0:00:01
     |████████████████████████████████| 143kB 50.3MB/s eta 0:00:01
     |████████████████████████████████| 3.8MB 37.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 34.2MB/s eta 0:00:01
     |████████████████████████████████| 163kB 46.8MB/s eta 0:00:01
     |█████

     |████████████████████████████████| 122kB 45.4MB/s eta 0:00:01
ERROR: snowflake-snowpark-python 1.17.0 has requirement cloudpickle!=2.1.0,!=2.2.0,<=2.2.1,>=1.6.0; python_version < "3.11", but you'll have cloudpickle 3.0.0 which is incompatible.
ERROR: snowflake-connector-python 3.10.1 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.2.1 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.25.3 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
ERROR: botocore 1.34.106 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.2.1 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip uninstall cloudpickle -y 
!pip install cloudpickle==2.2.1

Uninstalling cloudpickle-3.0.0:
  Successfully uninstalled cloudpickle-3.0.0
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# !pip uninstall numpy -y
# !pip install numpy==1.5.1

In [ ]:
# !pip uninstall snowflake-snowpark-python -y
# !pip install snowflake-snowpark-python==1.5.1

In [2]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F

In [3]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
#     "authenticator": "externalbrowser", # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
} 

In [4]:
# Make a Snowpark Connection

################################################################################################################
#  You can also use the SnowSQL Client to configure your connection params:
#  https://docs.snowflake.com/en/user-guide/snowsql-install-config.html
#
#  >>> from snowflake.ml.utils import connection_params
#  >>> session = Session.builder.configs(connection_params.SnowflakeLoginOptions()
#  >>> ).create()   
#
#  NOTE: If you have named connection params then specify the connection name
#  Example:
#  
#  >>> session = Session.builder.configs(
#  >>> connection_params.SnowflakeLoginOptions(connection_name='connections.snowml')
#  >>> ).create()
#
#################################################################################################################

# Edit the connection.json before creating the session object below
# Create Snowflake Session object
# connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : ADITYASINGH
Role                        : "ADITYASINGH"
Database                    : "FIRST_DB"
Schema                      : "PUBLIC"
Warehouse                   : "FOSFOR_INSIGHT_WH"
Snowflake version           : 8.20.10
Snowpark for Python version : 1.17.0


In [ ]:
import numpy as np
import pandas as pd
import random
import string

from sklearn.datasets import make_regression
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.snowpark import Session

# Create a session with your preferred method
# session =

NUMERICAL_COLS = ["X1", "X2", "X3"]
CATEGORICAL_COLS = ["C1", "C2", "C3"]
FEATURE_COLS = NUMERICAL_COLS + CATEGORICAL_COLS
CATEGORICAL_OUTPUT_COLS = ["C1_OUT", "C2_OUT", "C3_OUT"]
FEATURE_OUTPUT_COLS = ["X1_FEAT_OUT", "X2_FEAT_OUT", "X3_FEAT_OUT", "C1_FEAT_OUT", "C2_FEAT_OUT", "C3_FEAT_OUT"]

# Create a dataset with numerical and categorical features
X, _ = make_regression(
    n_samples=1000,
    n_features=3,
    noise=0.1,
    random_state=0,
)
X = pd.DataFrame(X, columns=NUMERICAL_COLS)

def generate_random_string(length):
    return "".join(random.choices(string.ascii_uppercase, k=length))

categorical_feature_length = 2
categorical_features = {}
for c in CATEGORICAL_COLS:
    categorical_column = [generate_random_string(categorical_feature_length) for _ in range(X.shape[0])]
    categorical_features[c] = categorical_column

X = X.assign(**categorical_features)

features_df = session.create_dataframe(X)

# Fit a pipeline with OrdinalEncoder and MinMaxScaler on Snowflake
pipeline = Pipeline(
    steps=[
        (
            "OE",
            OrdinalEncoder(
                input_cols=CATEGORICAL_COLS,
                output_cols=CATEGORICAL_OUTPUT_COLS,
            )
        ),
        (
            "MMS",
            MinMaxScaler(
                input_cols=NUMERICAL_COLS + CATEGORICAL_OUTPUT_COLS,
                output_cols=FEATURE_OUTPUT_COLS,
            )
        ),
    ]
)

pipeline.fit(features_df)

# Use the pipeline to transform a dataset.
result = pipeline.transform(features_df)

In [7]:
from snowflake.ml.modeling.xgboost import XGBRegressor

In [10]:
xgboost_model = XGBRegressor(
    input_cols=FEATURE_COLS,
    label_cols=CATEGORICAL_OUTPUT_COLS,
    output_cols=FEATURE_OUTPUT_COLS
)

xgboost_model.fit(features_df)

# Use the model to make predictions.
predictions = xgboost_model.predict(features_df)
predictions[OUTPUT_COLS].show()

SnowparkSQLException: (1300) (1304): 01b4894b-0000-64e6-0000-576d06a7a48e: 000904 (42000): SQL compilation error: error line 1 at position 92
invalid identifier 'C1_OUT'

In [11]:
import pandas as pd
from sklearn.datasets import make_classification

from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session


In [12]:
FEATURE_COLS = ["X1", "X2", "X3", "X4", "X5", "X6"]
LABEL_COLS = ["Y"]
OUTPUT_COLS = ["PREDICTIONS"]

# Set up data.
X, y = make_classification(
    n_samples=40000,
    n_features=6,
    n_informative=4,
    n_redundant=1,
    random_state=0,
    shuffle=True,
)

X = pd.DataFrame(X, columns=FEATURE_COLS)
y = pd.DataFrame(y, columns=LABEL_COLS)

In [14]:
print(X)
print(y)

             X1        X2        X3        X4        X5        X6
0      1.342697 -0.647206  1.614209 -3.257509 -0.525833  1.150915
1     -1.116676  3.057113 -0.818561  1.458397  0.143306  0.794167
2     -0.386608 -1.669360  2.275258 -0.285974 -1.011781  1.415241
3      2.320792 -1.520405  0.518970  0.498914 -1.019967  0.052654
4     -1.082534  1.051277 -0.525338  1.941458  0.059289 -0.487503
...         ...       ...       ...       ...       ...       ...
39995  4.442924 -0.304902  0.063692 -0.551800 -1.296828 -0.996133
39996 -0.860829  1.522871 -2.181003 -0.156346  1.485009  0.736772
39997  2.190909 -0.536251 -0.378094 -0.783875 -0.213353 -0.834365
39998 -0.084504 -0.348913  1.767278 -2.652884 -0.347461 -0.881851
39999 -1.066430 -1.129243  1.358532 -0.850732 -0.146987  0.551301

[40000 rows x 6 columns]
       Y
0      1
1      1
2      0
3      0
4      1
...   ..
39995  0
39996  1
39997  0
39998  1
39999  0

[40000 rows x 1 columns]


In [7]:
import pandas as pd
from sklearn.datasets import make_classification

from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
from snowflake.snowpark import Session

# Create a session with your preferred method
# session =

FEATURE_COLS = ["X1", "X2", "X3", "X4", "X5", "X6"]
LABEL_COLS = ["Y"]
OUTPUT_COLS = ["PREDICTIONS"]

# Set up data.
X, y = make_classification(
    n_samples=40000,
    n_features=6,
    n_informative=4,
    n_redundant=1,
    random_state=0,
    shuffle=True,
)

X = pd.DataFrame(X, columns=FEATURE_COLS)
y = pd.DataFrame(y, columns=LABEL_COLS)

features_pandas = pd.concat([X, y], axis=1)
features_df = session.create_dataframe(features_pandas)

# Train an XGBoost model on snowflake.
xgboost_model = XGBRegressor(
    input_cols=FEATURE_COLS,
    label_cols=LABEL_COLS,
    output_cols=OUTPUT_COLS
)

xgboost_model.fit(features_df)

# Use the model to make predictions.
predictions = xgboost_model.predict(features_df)
predictions[OUTPUT_COLS].show()

ImportError: C extension: None not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.

In [1]:
!pip list | grep clou

cloudpickle                2.2.1      
You should consider upgrading via the 'pip install --upgrade pip' command.
